Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the \"License\")

In [ ]:
#@title License
# Licensed under the Apache License, Version 2.0 (the \"License\");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Load the Impressionist Datasets

## First, download it from Kaggle and upload it to Google drive. Then, load it using the cells below.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Single image to MPI example Colab

This Colab is part of code for the paper ___Single-view view synthesis with multiplane images___, and may be found at <br>https://github.com/google-research/google-research/tree/master/single_view_mpi.

The project site is at https://single-view-mpi.github.io/.

Choose __Run all__ from the Runtime menu to:
* set up the network and load our trained model,
* apply it to an RGB input to generate a 32-layer MPI,
* show individual MPI layers and synthesized disparity,
* render novel views from different camera positions,
* visualize the resulting MPI in an HTML-based viewer.


## Download library code, model weights, and example image.

In [2]:
!echo Fetching code from github...
!apt install subversion
!svn export --force https://github.com/google-research/google-research/trunk/single_view_mpi

!echo
!echo Fetching trained model weights...
!rm single_view_mpi_full_keras.tar.gz
!rm -rf single_view_mpi_full
!wget https://storage.googleapis.com/stereo-magnification-public-files/models/single_view_mpi_full_keras.tar.gz
!tar -xzvf single_view_mpi_full_keras.tar.gz

!echo
!echo Fetching example image...
!rm -f input.png
!wget https://single-view-mpi.github.io/mpi/7/input.png


Fetching code from github...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libsvn1 amd64 1.9.7-4ubuntu1 [1,183 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 sub

## Set up the model


In [3]:
pip install -r single_view_mpi/requirements.txt

     |████████████████████████████████| 516.2 MB 4.1 kB/s 
     |████████████████████████████████| 1.0 MB 37.7 MB/s 
     |████████████████████████████████| 454 kB 25.2 MB/s 
     |████████████████████████████████| 3.0 MB 34.3 MB/s 
     |████████████████████████████████| 2.9 MB 22.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Fo

In [4]:
import tensorflow as tf

from single_view_mpi.libs import mpi
from single_view_mpi.libs import nets

input = tf.keras.Input(shape=(None, None, 3))
output = nets.mpi_from_image(input)

model = tf.keras.Model(inputs=input, outputs=output)
print('Model created.')
# Our full model, trained on RealEstate10K.
model.load_weights('single_view_mpi_full_keras/single_view_mpi_keras_weights')
print('Weights loaded.')


Model created.
Weights loaded.


# Load the samples of data we want to analyze

In [16]:
import os
FILE_PATH = "gdrive/MyDrive/cs 231a/impressionist_landscapes_resized_1024/"
os.chdir(FILE_PATH)

FileNotFoundError: ignored

In [11]:
paintings = os.listdir()
paintings[0]

'guy-rose_a-carmel-pine-1918.jpg'

In [12]:
len(paintings)

1331

In [22]:
import random
sample_sz = 100
random_sample = random.sample(paintings, sample_sz)
random_sample

['georges-seurat_the-river-seine-at-la-grande-jatte-1888.jpg',
 'piet-mondrian_wood-with-beech-trees.jpg',
 'claude-monet_landscape-at-port-villez-1.jpg',
 'henri-edmond-cross_untitled.jpg',
 'giuseppe-de-nittis_lungo-l-ofanto.jpg',
 'eugene-boudin_near-antibes-1893.jpg',
 'camille-pissarro_the-garden-at-pontoise-1877.jpg',
 'camille-pissarro_the-fields-of-eragny-the-apple-tree-1894.jpg',
 'robert-lewis-reid_landscape-with-waterfall-1890.jpg',
 'john-singer-sargent_corfu-the-terrace-1909.jpg',
 'maximilien-luce_terril-de-charbonnage-1896.jpg',
 'maximilien-luce_the-shepherd-observing-the-sea-1906.jpg',
 'pierre-auguste-renoir_cliffs-1883.jpg',
 'camille-pissarro_road-in-port-maryl-1872.jpg',
 'camille-pissarro_white-horse-on-a-farm.jpg',
 'pierre-auguste-renoir_cagnes.jpg',
 'piet-mondrian_at-the-amstel-sun.jpg',
 'alfred-sisley_on-the-banks-of-the-river-loing.jpg',
 'claude-monet_cliffs-at-pourville-in-the-fog.jpg',
 'gustave-caillebotte_the-yerres-rain.jpg',
 'jose-maria-velasco_p-rf

# Calculate PSNR and SSIM

Sourced from: https://cvnote.ddlee.cc/2019/09/12/psnr-ssim-python

In [23]:
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    # img1 = img1.astype(np.float64)
    # img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    # img1 = img1.astype(np.float64)
    # img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [27]:
# calculate SSIM and PSNR for samples
num_layers = 32
ssims = np.zeros((sample_sz, num_layers))
psnrs = np.zeros((sample_sz, num_layers))
for i in range(sample_sz):
  if (i % 10 == 0):
    print(i)
  # Input image
  inputfile = random_sample[i]
  input_rgb = tf.image.decode_image(tf.io.read_file(inputfile), dtype=tf.float32)

  # Generate MPI
  layers = model(input_rgb[tf.newaxis])[0]

  # Note: this only selects RGB layers and not output

  for j in range(len(layers)):
    ssims[i][j] = calculate_ssim(input_rgb.numpy(), layers[j][:, :, 0:3].numpy())
    psnrs[i][j] = calculate_psnr(input_rgb, layers[j][:, :, 0:3])

print("mean ssim:")
print(ssims.mean())
print("mean psnr:")
print(psnrs.mean())

0
10
20


KeyboardInterrupt: ignored

In [28]:
print("mean ssim:")
print(ssims.mean())
print("mean psnr:")
print(psnrs.mean())

mean ssim:
0.2034299873933196
mean psnr:
inf


In [29]:
psnrs

array([[ 75.46670767,  75.47785671,  75.4981092 , ..., 115.01969813,
        116.52811882,          inf],
       [ 79.23494013,  79.26290074,  79.31002571, ..., 111.86418   ,
        113.04810486,          inf],
       [ 75.8513435 ,  75.86082165,  75.87739334, ..., 115.42871549,
        116.74248132,          inf],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [31]:
ssims

array([[0.99990016, 0.9999004 , 0.99990064, ..., 1.        , 1.        ,
        1.        ],
       [0.99993515, 0.99993545, 0.99993616, ..., 1.        , 1.        ,
        1.        ],
       [0.99990267, 0.9999029 , 0.9999029 , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])